# 強化学習と線型計画法

* 参考文献: [強化学習 (機械学習プロフェッショナルシリーズ) , 66ページ付近　](https://www.amazon.co.jp/%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%83%95%E3%82%A7%E3%83%83%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E6%A3%AE%E6%9D%91-%E5%93%B2%E9%83%8E/dp/4065155916/ref=asc_df_4065155916/?tag=jpgo-22&linkCode=df0&hvadid=310429813636&hvpos=&hvnetw=g&hvrand=6867034787001615408&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1009224&hvtargid=pla-675730625220&psc=1&th=1&psc=1)
* [Reinforcement Learning via Fenchel-Rockafellar Duality](https://arxiv.org/abs/2001.01866)

強化学習が扱う最適方策の導出（プランニング問題）は線形計画問題としても定式化できます。
線形計画問題の双対問題を考えることで、通常よりも効率の良い解法が見つかる場合があります（ロバストMDPの解析などでも出てきます）。

**表記**（[RL_utils.ipynb](RL_utils.ipynb)参照）

MDPを次で定義します。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. 遷移確率行列: $P\in \mathbb{R}^{SA\times S}$
4. 報酬行列: $r\in \mathbb{R}^{S\times A}$
5. 割引率: $\gamma \in [0, 1)$
6. 初期状態: $\mu \in \mathbb{R}^{S}$

* 内積の記法: $f_1, f_2 \in \mathbb{R}^{S\times A}$に対して、$\langle f_1, f_2 \rangle = (\sum_{a\in A} f_1(s, a)f_2(s, a))_s \in \mathbb{R}^S$とします。これは方策についての和を省略するときなどに便利です。例えば$\langle \pi, q_\pi\rangle = v_\pi$です。
* 方策行列（$\Pi^\pi \in \mathbb{R}^{S\times SA}$）：$\langle \pi, q\rangle$を行列で書きたいときに便利。
    * $\Pi^\pi(s,(s, a))=\pi(a \mid s)$ 
    * $\Pi^\pi q^\pi = \langle \pi, q^\pi \rangle = v^\pi$が成立。
* 遷移確率行列（$P^\pi \in \mathbb{R}^{SA\times SA}$）: 次の状態についての方策の情報を追加したやつ。
    * $P^\pi = P \Pi^\pi$
    * Q値を使ったベルマン期待作用素とかで便利。$q^\pi = r + \gamma P^\pi q^\pi$が成立。
    * $(I - \gamma P^\pi)^{-1}r = q^\pi$が成立する。
* 割引訪問頻度（$d^\pi_\mu \in \mathbb{R}^{SA}$）：S, Aについての割引累積訪問頻度
    * ${d}^\pi_\mu (s, a) = \pi(a|s) \sum_{s_0} \mu(s_0) \sum_{t=0}^\infty \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$がで定義される。
    * $d^\pi_\mu = \mu \Pi^\pi (I - \gamma P^\pi)^{-1} = \mu (I - \gamma \bar{P}^\pi)^{-1} \Pi^\pi$が成立。
    * $d^\pi_\mu = \mu \Pi^\pi + \gamma d^\pi_\mu P^\pi$が成立。動的計画法のように解ける。
* 初期状態からの方策の価値：$\rho(\pi)= \mu \Pi^\pi Q^\pi=d^\pi_\mu r$

線形計画問題の導出のために、価値関数についての上界と下界の定理を見てみましょう。

---

**補題：価値関数の上界と下界**

最適価値関数の上界と下界には次の関係が成立します

* 上界：$v \geq (B v)$のとき、$v \geq v^*$
* 下界：$v \leq (B v)$のとき、$v \leq v^*$

証明は簡単です。ベルマン作用素の単調性から、

$$v \geq (B v)  \geq (B^k v)  \geq \dots \geq v^*$$

が成り立ちます。下界も同様です。
また、これは期待ベルマン作用素についても成立します。

* 上界：$v \geq (B_\pi v) $のとき、$v \geq v^\pi$
* 下界：$v \leq (B_\pi v) $のとき、$v \leq v^\pi$

---

これを使って**方策評価**と**方策最適化**の両方について、主問題と双対問題を見てみましょう。


## 方策評価の主問題と双対問題

参考：[Reinforcement Learning via Fenchel-Rockafellar Duality](https://arxiv.org/abs/2001.01866)

初期状態についての方策の評価を考えます。主問題として次の形式を考えましょう。

$$
\begin{aligned}
\rho(\pi)=\min _q & \;\mu \Pi^\pi q \\
\text { s.t. } & \;q \geq r+\gamma P^\pi q
\end{aligned}
$$

変形して、

$$
\begin{aligned}
\rho(\pi)=\min _q & \;\mu \Pi^\pi q \\
\text { s.t. } & \; (I - \gamma P^\pi) q - r \geq 0
\end{aligned}
$$


ここで、[CVX_convex_functions.ipynb](CVX_coFenchel-Rockas.ipynb)の線形計画問題における変換を思い出しましょう。
線形計画問題については次のような変形が成り立ちます：

---

**ラグランジアンによる変形**

上の方策評価の問題は、$b= \mu\Pi^\pi$、$A = (I - \gamma P^\pi)$、$c=r$とおけば、
$$
\min _y \langle b, y \rangle \text { s.t. } A y \geq c
$$
と等価です。よって、ラグランジアンによる変形を考えると、この問題は

$$
\max _{x \geq 0}\langle c, x\rangle \quad \text { s.t. }\quad A_* x=b
$$

と等価です。

---

よって、上の方策評価の問題は

$$
\rho(\pi) = 
\max _{d \geq 0}\langle r, d\rangle \quad \text { s.t. }\quad d(I - \gamma P^\pi)=\mu\Pi^\pi
$$
であり、変形して

$$
\rho(\pi) = 
\max _{d \geq 0}\langle r, d\rangle \quad \text { s.t. }\quad d = \mu\Pi^\pi + \gamma d P^\pi
$$

が成立します。これをPythonで再現してみましょう。

## 最適価値関数の主問題と双対問題

* 参考文献: [強化学習 (機械学習プロフェッショナルシリーズ) , 66ページ付近　](https://www.amazon.co.jp/%E5%BC%B7%E5%8C%96%E5%AD%A6%E7%BF%92-%E6%A9%9F%E6%A2%B0%E5%AD%A6%E7%BF%92%E3%83%97%E3%83%AD%E3%83%95%E3%82%A7%E3%83%83%E3%82%B7%E3%83%A7%E3%83%8A%E3%83%AB%E3%82%B7%E3%83%AA%E3%83%BC%E3%82%BA-%E6%A3%AE%E6%9D%91-%E5%93%B2%E9%83%8E/dp/4065155916/ref=asc_df_4065155916/?tag=jpgo-22&linkCode=df0&hvadid=310429813636&hvpos=&hvnetw=g&hvrand=6867034787001615408&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1009224&hvtargid=pla-675730625220&psc=1&th=1&psc=1)

---

**主問題**

上の補題から、$v^*$は$v \geq v^*$を満たす関数$v$の下界になります。
これをつかうと、$v^*$を求める問題は、不等式制約$v \geq (B v)$を満たす$v$のうち最小になる$v$によって近似することができます。
関数$v$の大小を測る指標として$v$の重み付き和$w^Tv$を導入すると、次の線形計画問題が考えられます：

$$
\begin{aligned}
&\mathrm{minimize}_{v \in \mathbb{R}^{S}} \; w^T v\\
&\text{ subject to } v \geq \Pi\left(r + \gamma (P v) \right)
\end{aligned}
$$

$\Pi$にはmax作用素が入ってしまっていますが、次のように変形して、不等式を$\forall s, a \in {S}\times {A}$で評価すれば大丈夫です。

$$
\begin{aligned}
&\mathrm{minimize}_{v \in \mathbb{R}^{S}} \; w^T v\\
&\text{ subject to } (\tilde{I} - \gamma P)v \geq r
\end{aligned}
$$

ここで$\tilde{I}:=[I_{|S|}, I_{|S|}, \dots] \in \mathbb{R}^{{SA}\times S}$としました。

---

---

**双対問題**

主問題を双対化してみましょう。
つまり、 目的関数を制約条件に、そして制約条件を目的変数に逆転させることで、最適化問題を最定式化します。

一応線形計画問題の強双対性については触れておきます

* 線形計画問題が有界な最適解$x^*$をもつならば、双対問題も有界の最適解$\nu^*$をもち、互いの目的案数の最適値は一致する。$c^Tx^* = b^T \nu^*$
* 主問題の目的関数が非有界ならば、双対問題は実行不可能である。つまり、任意の$M\in \mathbb{R}$について、主問題の目的関数の値$f(x)$を$M$より大きくできるような実行可能解$x$が存在する場合、双対問題は実行不可能である。

つまり、プランニングでは双対問題を変わりに解いても、主問題と同じ解が得られます。

主問題をラグランジュの未定乗数法を使って双対化すると

$$
\begin{aligned}
&\mathrm{maximize}_{x \in \mathbb{R}^{SA}} \; x^T r \\
&\text{ subject to } x^T(\tilde{I} - \gamma P) = w, \; x \geq 0
\end{aligned}
$$

になります。

---

双対問題の特性を調べるために、割引訪問頻度を導入します。

* 割引訪問頻度（$d^\pi_\mu \in \mathbb{R}^{SA}$）：S, Aについての割引累積訪問頻度
    * $d^\pi_\mu = \mu \Pi^\pi (I - \gamma P^\pi)^{-1} = \mu (I - \gamma \bar{P}^\pi)^{-1}$
    * ${d}^\pi_\mu (s, a) = \pi(a|s) \sum_{s_0} \mu(s_0) \sum_{t=0}^\infty \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$が成立する。

これを使うと、$(d^\pi_w)^T r = w^T v^\pi$ であることが簡単にわかります。

ちなみに$\pi, \pi^\prime\in \Pi$が$\pi(a|s)=\pi^\prime(a|s)$のとき、かつそのときに限り、$d^\pi_w(s, a)=d^{\pi^\prime}_w(s, a)$になります。（証明は教科書の補題2.11参照）
つまり、$\pi$と割引訪問頻度$d^\pi_w$は一対一対応しています。

さらに、$d^\pi_w$については次の命題が成立します。

---

* 任意の方策$\pi\in \Pi$の割引訪問頻度は双対問題の実行可能解$x$になる。

まずこれを示します。定義から明らかに$d^\pi_w \geq 0$です。また、

$$
\begin{aligned}
&\sum_{s, a} \gamma P(s^\prime|s, a)d^\pi_w(s, a)\\
&=\sum_{s_0} w(s_0) \sum_{s, a} \gamma P(s^\prime|s, a)\sum^\infty_{t=0} \gamma^t \mathrm{Pr}(S_t=s, A_t=a| S_0=s_0, M(\pi))\\
&=\sum_{s_0} w(s_0) \sum^\infty_{t=0} \gamma^{t+1} \mathrm{Pr}(S_{t+1}=s^\prime| S_0=s_0, M(\pi))\\
&=\sum_{s_0} w(s_0) \sum^\infty_{t=1} \gamma^{t} \mathrm{Pr}(S_{t}=s^\prime| S_0=s_0, M(\pi))\\
&=\sum_{s_0} w(s_0) \left(\sum^\infty_{t=1} \gamma^{t} \mathrm{Pr}(S_{t}=s^\prime| S_0=s_0, M(\pi)) - \mathrm{Pr}(S_{0}=s^\prime| S_0=s_0, M(\pi)) \right)\\
&=\sum_{s_0} w(s_0) \sum^\infty_{t=1} \gamma^{t} \mathrm{Pr}(S_{t}=s^\prime| S_0=s_0, M(\pi)) - w(s^\prime)\\
&=\sum_{a^\prime} d^\pi_w (s^\prime, a^\prime) - w(s^\prime)
\end{aligned}
$$

なので、割引訪問頻度は実行可能解になってます。

（たぶん
$d^\pi_w(s, a) = \sum_{s_0 \in \mathcal{S}}w(s_0) \sum^\infty_{t=0} \gamma^t \mathrm{Pr}(S_t=s, A_t=a| S_0=s_0, M(\pi))$
は$(I - \gamma P)^{-1}$に$w(s_0)$の要素が入ってるので、$(I - \gamma P)^T x$の$x$に代入すると$(I - \gamma P)$がキャンセルされて$w$だけ残るはず。
行列をいい感じに書き直したらもっとキレイにかけるかも。
[RL_utils.ipynb](RL_utils.ipynb)の行列を使えばキレイに書けそう！
）

また、次の命題も成立します。

* ある関数$x\in \mathbb{R}^{\mathcal{S}\times \mathcal{A}}$が双対問題の実行可能解のとき、確率的方策$\pi_x\propto x$の割引訪問頻度$d^{\pi_x}_w$は関数$x$と一致する。つまり、$d^{\pi_x}_w=x$

こっちは省略しますが、行列をいじるとすぐに出てきます。
これはつまり、実行可能解から方策に変換するやり方と、その方策の割引訪問頻度から実行可能解を復元するやり方を説明しています。

---

**主問題と双対問題の関係**

上の命題を踏まえて、次の作用素を考えましょう。

* 実行可能解から方策に変換する作用素：$\mathcal{P}$
* 方策から割引訪問頻度に変換する作用素：$\mathcal{D}$

このとき、$\mathcal{P}$と$\mathcal{D}$は全単射であり、一対一対応します。
つまり、

$$\mathcal{P}(x) = \mathcal{D}^{-1}(x) \text{ かつ } \mathcal{D}(\pi) = \mathcal{P}^{-1}(\pi)$$
が成立します。

以上より、$\sum_{s\in\mathcal{S}}\sum_{a\in \mathcal{A}}x(s, a) r(s, a)$は$\sum_{s\in\mathcal{S}} w(s) v^{\pi_x}(s)$と同じです。
よって双対問題は最大の価値関数を探す問題と同じなんですね。
さらに、重み関数$w$が０より大きければ、その選択にかかわらず、主問題の最適解は最適価値関数と一致します。

以上の話をPythonで実験してみましょう。


---

**主問題**

上の補題から、$v^*$は$v \geq v^*(s)$を満たす関数$v$の下界になります。
これをつかうと、$v^*$を求める問題は、不等式制約$v(s) \geq (B v) (s)$を満たす$v$のうち最小になる$v$によって近似することができます。
関数$v$の大小を測る指標として$v(s)$の重み付き和$\langle w, v\rangle := \sum_{s\in \mathcal{S}}w(s)v(s)$を導入すると、次の線形計画問題が考えられます：

$$
\begin{aligned}
&\mathrm{minimize}_{v \in \mathbb{R}^{\mathcal{S}}} \; w^T v \\
&\text{ subject to } v \geq r^\pi +\gamma P^\pi v
\end{aligned}
$$

ここで、次の記法を使っています。
* $r^\pi(s) = \sum_{a\in \mathcal{A}} \pi(s, a)r(s, a)$
* $P^\pi(s^\prime|s) = \sum_{a\in \mathcal{A}} \pi(s, a)P(s^\prime | s, a)$

$w$を初期分布$\mu$に対応させると、目的関数は期待リターンになることに注意しましょう。

---

---

**双対問題**

主問題を少し変形すると、

$$
\begin{aligned}
&\mathrm{minimize}_{v \in \mathbb{R}^{\mathcal{S}}} \; w^T v \\
&\text{ subject to } (I - \gamma P^\pi) v \geq r^\pi
\end{aligned}
$$

になります。
この制約付き最適化問題を双対化します。
つまり、 目的関数を制約条件に、そして制約条件を目的変数に逆転させることで、最適化問題を最定式化します。
（長くなるので省略しますが、詳細は教科書のA.3を参照してください。）

一応線形計画問題の強双対性については触れておきます

* 線形計画問題が有界な最適解$x^*$をもつならば、双対問題も有界の最適解$\nu^*$をもち、互いの目的案数の最適値は一致する。$c^Tx^* = b^T \nu^*$
* 主問題の目的関数が非有界ならば、双対問題は実行不可能である。つまり、任意の$M\in \mathbb{R}$について、主問題の目的関数の値$f(x)$を$M$より大きくできるような実行可能解$x$が存在する場合、双対問題は実行不可能である。

つまり、プランニングでは双対問題を変わりに解いても、主問題と同じ解が得られます。

主問題をラグランジュの未定乗数法を使って双対化すると

$$
\begin{aligned}
&\mathrm{maximize}_{x \in \mathbb{R}^{\mathcal{S}}} \; x^T r^\pi \\
&\text{ subject to } (I - \gamma P^\pi) x = w, \; x \geq 0
\end{aligned}
$$

になります。

---

双対問題の特性を調べるために、割引訪問頻度を導入します。

* $\pi$についての割引状態訪問頻度：$d^\pi_w(s) := \sum_{s_0 \in \mathcal{S}}w(s_0) \sum^\infty_{t=0} \gamma^t \mathrm{Pr}(S_t=s| S_0=s_0, M(\pi))$

これを使うと、$\sum_s d^\pi_w(s) r^\pi(s) = \sum_s w(s) v^\pi(s)$ であることが簡単にわかります。

ちなみに$\pi, \pi^\prime\in \Pi$が$\pi(a|s)=\pi^\prime(a|s)$のとき、かつそのときに限り、$d^\pi_w(s)=d^{\pi^\prime}_w(s)$になります。（証明は教科書の補題2.11からすぐに成り立ちます。） -->


In [8]:
import numpy as np
from typing import NamedTuple


S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = np.arange(S)  # 状態集合
A_set = np.arange(A)  # 行動集合
gamma = 0.8  # 割引率

# 報酬行列を適当に作ります
rew = np.random.rand(S, A)
assert rew.shape == (S, A)

# 遷移確率行列を適当に作ります
P = np.random.rand(S*A, S)
P = P / np.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(S, A, S)
np.testing.assert_almost_equal(P.sum(axis=-1), 1)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: np.array  # 状態集合
    A_set: np.array  # 行動集合
    gamma: float  # 割引率
    horizon: int  # エフェクティブホライゾン
    rew: np.array  # 報酬行列
    P: np.array  # 遷移確率行列

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


horizon = int(1 / (1 - gamma))
mdp = MDP(S_set, A_set, gamma, horizon, rew, P)

In [9]:
import jax
from functools import partial
import jax.numpy as jnp

# 動的計画法で最適価値関数を計算します。

@partial(jax.jit, static_argnames=("S", "A"))
def _compute_optimal_V(mdp: MDP, S: int, A: int):

    def backup(optimal_Q):
        max_Q = optimal_Q.max(axis=1)
        next_v = mdp.P @ max_Q
        assert next_v.shape == (S, A)
        return mdp.rew + mdp.gamma * next_v
    
    optimal_Q = jnp.zeros((S, A))
    body_fn = lambda i, Q: backup(Q)
    Q = jax.lax.fori_loop(0, mdp.horizon + 1000, body_fn, optimal_Q)
    return Q.max(axis=-1)

compute_optimal_V = lambda mdp: _compute_optimal_V(mdp, mdp.S, mdp.A)

optimal_V_by_DP = compute_optimal_V(mdp)

In [10]:
from scipy.optimize import linprog
# 重みはランダムに設定してみます
w = np.random.rand(S)

P = mdp.P.reshape(S*A, S)
I = np.eye(S).reshape(S, 1, S)
tI = np.tile(I, [1, A, 1]).reshape(S*A, S)

assert P.shape == tI.shape

# 主問題を解きます
# Ax < b の不等式用です
I_gP = - (tI - mdp.gamma * P)
r = - mdp.rew.reshape(-1)
lin_res = linprog(w, A_ub=I_gP, b_ub=r)

optimal_V_by_LP = lin_res.x
DP_fn = optimal_V_by_DP @ w

print("DPとLPの解の差：", np.abs(optimal_V_by_LP - optimal_V_by_DP).max())
print("DPとLPの目的関数の差", (DP_fn - lin_res.fun))

DPとLPの解の差： 4.7683716e-07
DPとLPの目的関数の差 1.9073486e-06


動的計画法と主問題の結果はほぼ一致していますね。双対問題はどうでしょうか？

In [11]:
# 双対問題を解きます
# Ax = b の等式用です
r = mdp.rew.reshape(-1)
I_gP = (tI - mdp.gamma * P).T
dual_res = linprog(-r, A_eq=I_gP, b_eq=w, bounds=(0, None))  # Maximumなのでマイナスをつけます

print("LPとDUALの目的関数の差", (lin_res.fun + dual_res.fun))  # Maximumなので和で計算します

LPとDUALの目的関数の差 2.4868995751603507e-14


DPとLPとDUALはほぼ同じ答えにたどり着いていますね。よかったよかった。

TODO: 今回の表記は結構めんどうなので、もっと単純にかける気がする。